In [10]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
from pytz import timezone
from datetime import datetime

def config(filename='config.ini', section='aiaqua'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # print(Path(os.path.dirname(os.path.realpath(__file__)) + '/' + filename))
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(
            'Section {0} not found in the {1} file'.format(section, filename))
    return db

params = config()
conn = psycopg2.connect(**params)
cursor = conn.cursor()    

tz = timezone('Asia/Ho_chi_Minh')
loc_dt = tz.localize(datetime.now())
created_date = loc_dt.strftime("%Y-%m-%d %H:%M:%S")
snapshot_date = loc_dt.strftime("%Y-%m-%d")

df = pd.read_excel('Sensor data for 30 cm.xlsx')
df.columns = ['date_and_time','temperature_c','ph','turbidity_ntu']
columns = df.columns

tmp_df = '/home/ailab/AIFace/AIAquaKMOU/backend/tmp/temp-{}.csv'.format(str(datetime.now().strftime("%Y%m%d%H%M%S%f")))
df.to_csv(tmp_df, index=False, header=True)
sql_copy = """
    COPY %s(%s) 
    FROM '%s' DELIMITER ',' CSV HEADER encoding 'utf-8'; 
"""% ('sensor_data',','.join(columns), tmp_df)

cursor.execute(sql_copy)
conn.commit()   

cursor.close()  

print(df)

            date_and_time  temperature_c    ph  turbidity_ntu
0     2020-01-15 16:00:35          20.99  7.81            197
1     2020-01-15 16:01:33          20.99  7.81            197
2     2020-01-15 16:02:33          20.98  7.81            197
3     2020-01-15 16:03:22          20.98  7.81            197
4     2020-01-15 16:04:26          20.98  7.81            197
...                   ...            ...   ...            ...
9618  2020-01-22 16:21:01          21.32  7.49            212
9619  2020-01-22 16:22:09          21.32  7.49            212
9620  2020-01-22 16:23:16          21.34  7.49            212
9621  2020-01-22 16:24:23          21.34  7.50            212
9622  2020-01-22 16:25:32          21.31  7.51            212

[9623 rows x 4 columns]
